    Fundamentos de Data Science – Práctica para el Examen

Contexto:

La tarea es limpiar y explorar los datos para una compañía de tecnología emergente que desarrolla aplicaciones móviles. La empresa quiere mejorar la experiencia del usuario y aumentar la retención de usuarios en sus aplicaciones. Han recolectado datos sobre el uso de sus aplicaciones y quieren entender mejor cómo los usuarios interactúan con sus productos.

In [18]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

In [2]:
# Configuramos la semilla para reproducibilidad
np.random.seed(42)

# Generamos los datos sintéticos
user_id = range(1, 301)
app_version = np.random.choice(['1.0', '1.1', '1.2', '1.3'], 300)
platform = np.random.choice(['Android', 'iOS'], 300)
session_duration = np.random.randint(1, 180, 300)
number_of_sessions = np.random.randint(1, 20, 300)
country = np.random.choice(['USA', 'Canada', 'Mexico', 'UK', 'Germany', 'France', 'Spain', 'Italy'], 300)
user_feedback = np.random.randint(1, 6, 300)


In [4]:
# Creamos el DataFrame
data = pd.DataFrame({
    'user_id': user_id,
    'app_version': app_version,
    'platform': platform,
    'session_duration': session_duration,
    'number_of_sessions': number_of_sessions,
    'country': country,
    'user_feedback': user_feedback
})
data.to_csv("app_mobile.csv", index=False)

In [ ]:
df = pd.read_csv('../data/app_mobile.csv')

df.head(5)


# 1) Limpieza de Datos:

El primer paso en este proceso es usar Python para limpiar los datos y dejarlos listos para el análisis. Deben:

## Identificar y abordar cualquier valor duplicado.

In [ ]:
# Detección y eliminación de valores duplicados

duplicados = df.duplicated()
print(f"Se han detectado {duplicados.sum()} duplicados")


## Identificar y abordar cualquier dato que falte en este conjunto de datos. 
Trátenlos de forma adecuada. Incluyan una breve descripción en el método que usan para tratar con los valores que faltan junto con una justificación para el uso de ese método.

In [ ]:
qsna=df.shape[0]-df.isnull().sum(axis=0)
qna=df.isnull().sum(axis=0)
ppna=round(100*(df.isnull().sum(axis=0)/df.shape[0]),2)
aux= {'datos sin NAs en q': qsna, 'Na en q': qna ,'Na en %': ppna}
na=pd.DataFrame(data=aux)
na.sort_values(by='Na en %',ascending=False)

## Asegurarse de que todas las columnas coincidan con los tipos de datos enumerados en el diccionario de datos.

In [ ]:
df.info()

In [ ]:
data_type = {
    'platform' : 'category',
    'country' : 'category'
}
df = df.astype(data_type)
print(df.dtypes)

## Identificar y abordar cualquier inconsistencia en los valores categóricos (ejemplo: android, Android, ANDROID).

In [ ]:
columnas_string = df.select_dtypes(include=['category'])

# Mostrar valores únicos de cada columna de tipo cadena
for col in columnas_string.columns:
    print(f"Valores únicos en la columna '{col}':")
    print(df[col].unique())
    print()

In [ ]:
# Convertir las cadenas en minúsculas
def cadena_minus(cadena):
    if isinstance(cadena , str):
        cadena = cadena.lower()
    return cadena
# aplicar la funcion a las columnas tipo cadena
for col in columnas_string:
    df[col] = df[col].apply(cadena_minus)
# Verificar los cambios
for col in columnas_string:
    print(f"\nValores únicos después de limpieza en '{col}':")
    print(df[col].unique())

## Identificar y abordar cualquier punto de datos inapropiados o inusuales (ejemplo: tiempo de uso de 10000 horas en una semana).

In [ ]:
verif_columnas = ['app_version', 'session_duration', 'number_of_sessions', 'user_feedback']
df[verif_columnas].describe()

### No se verifica datos anomalos

# 2) Exploración de Datos:

## Dos tipos diferentes de visualizaciones exploratorias univariantes. 

Cada visualización debe incluir una breve interpretación dentro del archivo de código.

In [ ]:
# Promedio de sesiones por tipo de sistema operativo por plataforma
os_type = df['platform'].value_counts()
session_duration = df['session_duration'].mean()
# creamos el grafico
plt.figure(figsize=(8, 4))
plt.bar(os_type.index, session_duration, color='skyblue')
plt.title('Promedio de sesiones por tipo de sistema operativo')
plt.xlabel('Tipo de OS')
plt.ylabel('Promedio de duracion de sesiones')
plt.xticks(rotation=0)
plt.show()

In [ ]:
# Agrupar el número de sesiones promedio por nivel de feedback
sessions_mean = df.groupby('user_feedback')['number_of_sessions'].mean()

# Graficar
sessions_mean.plot(kind='line', marker='o', color='teal')
plt.xlabel('User Feedback')
plt.ylabel('Promedio de numero de sesiones')
plt.title('Promedio de numero de sesiones por nivel de feedback')
plt.show()

## Dos tipos diferentes de visualizaciones exploratorias multivariantes. 
Cada visualización debe incluir una breve interpretación dentro del archivo de código.

In [ ]:
# Promedio de sesiones por tipo de sistema operativo por plataforma
feedback_mean = df.groupby(['number_of_sessions', 'platform'])['user_feedback'].mean().unstack()

# Graficar
feedback_mean.plot(kind='line', marker='o')
plt.xlabel('Number of Sessions')
plt.ylabel('Average User Feedback')
plt.title('Average User Feedback per Session by Platform')
plt.show()

In [ ]:
# Calcular el promedio de sesiones por calificación y plataforma
sessions_platform = df.groupby(['user_feedback', 'platform'])['number_of_sessions'].mean().unstack()

# Configurar el gráfico de barras
bar_width = 0.35  # Ancho de las barras
labels = sessions_platform.index  # Calificaciones de feedback
x = np.arange(len(labels))  # Posiciones en el eje x

# Crear el gráfico de barras
fig, ax = plt.subplots()

# Crear las barras para Android e iOS
bars1 = ax.bar(x - bar_width/2, sessions_platform['android'], bar_width, label='Android')
bars2 = ax.bar(x + bar_width/2, sessions_platform['ios'], bar_width, label='iOS')

# Etiquetas y título
ax.set_xlabel('User Feedback')
ax.set_ylabel('Promedio de numero de sesiones')
ax.set_title('Promedio de numero de sesiones por user feedback y plataforma')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

# Mostrar el gráfico
plt.show()